In [4]:
import os, sys
import importlib
import torch

import numpy as np

from utils import load_torch, load

In [6]:
importlib.reload(load_torch)

data_folder = "data/simplified/"
access_file_generator = map(lambda x : data_folder + x + '.csv', load.classes)
trainGenerator = load_torch.ImageLoader(load.classes_1, data_folder)

In [13]:
gen = trainGenerator.__iter__()

In [14]:
%matplotlib inline
X, y = next(gen)
print('X.shape:', X.shape)
print('y.shape:', y.shape)

X.shape: torch.Size([16, 1, 32, 32])
y.shape: torch.Size([16])


# Building PyTorch CNN Model

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag
import torch.optim as optim


class LeNet(nn.Module):
    # Here we define our network structure
    def __init__(self, cdim=8):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3).double() 
        self.conv2 = nn.Conv2d(6,16, 3).double()
        self.conv3 = nn.Conv2d(16,32,3).double()
        self.conv4 = nn.Conv2d(32,64,3).double()
        self.fc1   = nn.Linear(256,128).double()
        self.fc2   = nn.Linear(128, 84).double() 
        self.fc3   = nn.Linear(84,cdim).double()
        
    # Here we define one forward pass through the network
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv4(x)), (2, 2))
        x = x.view(-1, int(self.num_flat_features(x)))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # Determine the number of features in a batch of tensors
    def num_flat_features(self, x): 
        size = x.size()
        return np.prod(size[1:])


net = LeNet(cdim=len(load.classes_1))

In [25]:
B     = 128              # Minibatch size
T     = 10               # Number of epochs
gamma = .001             # learning rate
rho   = .9               # momentum

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    net.parameters(),
    lr=gamma,
    momentum=rho
)

In [26]:
if torch.cuda.is_available():
    net = net.cuda()

    
accList, lossList = list(), list()
for epoch in range(T):
    running_loss, k   = 0.0, 0
    for img,lbl in trainGenerator:
        inputs = ag.Variable(img, requires_grad=False)
        labels = ag.Variable(lbl, requires_grad=False)
        
        if torch.cuda.is_available():
            inputs  = inputs.cuda()
            labels  = labels.cuda()
        
        # Initialize the gradiaent to zero
        optimizer.zero_grad()
        output = net(inputs)
        loss   = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss[0]
        k += 1
        if k % 10 == 9:
            ypred = net(inputs)
            acc   = 100 * np.mean(np.argmax(y,1) == ypred.data.numpy().T.argmax(axis=0))
            print('[%d, %5d] loss: %.3f\t acc: %.1f' % (epoch + 1, k + 1, running_loss/10, acc))
            accList.append(acc)
            lossList.append(running_loss/10)
            running_loss = 0.0
            

print('Finished Training')

RuntimeError: Given input size: (32 x 2 x 2). Calculated output size: (64 x 0 x 0). Output size is too small at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/THNN/generic/SpatialConvolutionMM.c:45